# 개요

원본 Backtranslation_of_SQL_queries.py은 몇 가지 이유로 오류 발생하고 실행안된다.

오류를 픽스하고 이를 ipynb 파일로 바꾸었다.


In [1]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 KB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [11]:

import openai
openai.api_key = "sk-..."

In [12]:
from typing import List, Union


In [67]:
N: int = 3
TEMPERATURE: float = 0.3
ENGINE: str = "text-davinci-003"
SQL_STARTS = "SELECT"
GENERATING_STOPS = ["#", ";"]


def get_candidates(
    prompt: str,
) -> List[str]:
    response = openai.Completion.create(
        prompt=prompt,
        engine=ENGINE,
        temperature=TEMPERATURE,
        max_tokens=150,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=GENERATING_STOPS,
        n=N,
    )

    responses = [SQL_STARTS + choice.text for choice in response.choices]
    return responses


def rindex(lst: List, value: str) -> int:
    try:
        return len(lst) - lst[::-1].index(value) - 1
    except ValueError:
        raise ValueError(f"Answer start token `{value}` not found in the eval template")


def eval_candidate(
    generated_sql: str,
    nl_query: str,
    eval_template: str,
    answer_start_token: str = "--",
) -> float:

    prompt = eval_template.format(generated_sql, nl_query)
    # print(prompt)

    response = openai.Completion.create(
        engine=ENGINE,
        prompt=prompt,
        temperature=0,
        max_tokens=0,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        logprobs=1,
        echo=True,
    )

    # print(response)
    answer_start = rindex(
        response["choices"][0]["logprobs"]["tokens"], answer_start_token
    )
    logprobs = response["choices"][0]["logprobs"]["token_logprobs"][answer_start + 1 :]
    return sum(logprobs) / len(logprobs)


def generate_sql(
    prompt_template: str,
    table_definitions: str,
    nl_query: str,
    eval_template: str,
    return_all_results: bool = False,
) :

    # prompt를 만들고
    prompt = prompt_template.format(table_definitions, nl_query, SQL_STARTS)
    # print(prompt)

    # 호출하여 SQL을 생성한다.
    generated_sqls = get_candidates(prompt)

    # 평가한다.
    candidates = []
    for generated_sql in generated_sqls:
        quality = eval_candidate(generated_sql, nl_query, eval_template)
        candidates.append((generated_sql, quality))

    # 소팅하고
    candidates.sort(key=lambda x: x[1], reverse=True)

    if return_all_results:
        return candidates

    # 최고 점수의 것 반환        
    return candidates[0][0]


In [68]:
nl_query: str = "Return the name of each department that had more than 10 employees in June 2021"
eval_template: str = "{};\n-- Explanation of the above query in human readable format\n-- {}"

table_definitions: str = """
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
"""

prompt_template: str = """
### Postgres SQL tables, with their properties:
#{}#
### {}
{}
"""

result = generate_sql(
    prompt_template,
    table_definitions,
    nl_query,
    eval_template,
)
print(result)


SELECT    d.name
FROM
    Department d
    INNER JOIN Employee e ON d.id = e.department_id
    INNER JOIN Salary_Payments s ON e.id = s.employee_id
WHERE
    s.date BETWEEN '2021-06-01' AND '2021-06-30'
GROUP BY
    d.name
HAVING
    COUNT(e.id) > 10
